# Find MBTA 
So far I haven't found the MBTA numbers in the cthru budget data, this notebook is to look harder for them or find a way to estimate using expenditures with the logic like "if the average agency uses x% of their budget, then solving backwards MBTA should have a budget of __ ". Or if there are numbers posted online somewhere

In [1]:
%load_ext autoreload
%autoreload 2

In [77]:
import pandas as pd
from sodapy import Socrata
import numpy as np
import textract
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
app_token = "2Qa1WiG8G4kj1vGVd2noK7zP0"
client = Socrata("cthru.data.socrata.com", app_token)

In [4]:
budget = client.get("fmz7-6ft9", where="fiscal_year >= 2012", limit=999999)

In [5]:
budget= pd.DataFrame(budget)

In [6]:
budget.shape

(19329, 24)

In [46]:
pd.set_option("display.max.rows", 163)
budget.groupby("department_name").size().sort_values()

department_name
merit rating board                                          1
registry of motor vehicles                                  2
massachusetts aeronautics commission                        6
office of the child advocate                                6
department of industrial accidents                          6
board conciliation & arbitration                            6
joint labor management commission                           6
educational quality & accountability                        6
teachers retirement board                                   7
department of labor                                         8
state reclamation board                                     8
division of labor relations                                 9
governors council                                          10
labor relations commission                                 10
mass cultural council                                      10
massachusetts office on disability                    

In [8]:
budget["department_name"]= budget["department_name"].str.lower()

In [9]:
budget[budget["department_name"].str.contains("transportation")==True]["department_name"].unique()

array(['massachusetts department of transportation',
       'executive office of transportation'], dtype=object)

In [10]:
mDOT_budget = budget[budget["department_name"] == "massachusetts department of transportation"]
mDOT_budget

,fiscal_year,fund_number,fund_name,cabinet_secretariat_name,department_code,department_name,appropriation_account_number,appropriation_account_name,beginning_balance_prior,original_enacted_budget,...,authorized_retained_revenue_1,retained_revenue_estimate,retained_revenue_collected,planned_savings_9c_spending,total_enacted_budget,total_available_for_spending,total_expenses,balance_forward_prior,unexpended_reverted,other_statutorily_authorized
303,2015,0103,Commonwealth Transportation Fund,MASSACHUSETTS DEPARTMENT OF TRANSPORTATION,DOT,massachusetts department of transportation,15956367,Performance and Asset Management Advisory Council,0,0,...,NaN,0,0,0,0,0,0,0,0,NaN
1515,2016,0163,Massachusetts Tourism Fund,MASSACHUSETTS DEPARTMENT OF TRANSPORTATION,DOT,massachusetts department of transportation,70080900,Massachusetts Office of Travel and Tourism,0,0,...,0,0,0,0,0,20000,20000,0,0,NaN
1528,2018,0103,Commonwealth Transportation Fund,MASSACHUSETTS DEPARTMENT OF TRANSPORTATION,DOT,massachusetts department of transportation,15956367,Performance and Asset Management Advisory Council,0,0,...,NaN,0,0,0,0,0,0,0,0,NaN
1619,2014,0010,General Fund,MASSACHUSETTS DEPARTMENT OF TRANSPORTATION,DOT,massachusetts department of transportation,21000015,Unified Carrier Registration Retained Revenue,0,0,...,NaN,0,0,0,0,33870.9,33870.9,0,0,NaN
2368,2017,0103,Commonwealth Transportation Fund,MASSACHUSETTS DEPARTMENT OF TRANSPORTATION,DOT,massachusetts department of transportation,15956367,Performance and Asset Management Advisory Council,0,0,...,NaN,0,0,0,0,0,0,0,0,NaN
2616,2018,0010,General Fund,MASSACHUSETTS DEPARTMENT OF TRANSPORTATION,DOT,massachusetts department of transportation,11030001,DEBT COLLECTION PAYMENTS,0,0,...,NaN,0,0,0,0,0,0,0,0,NaN
3177,2016,0010,General Fund,MASSACHUSETTS DEPARTMENT OF TRANSPORTATION,DOT,massachusetts department of transportation,44001000,DEPARTMENT OF PUBLIC WELFARE,0,0,...,0,0,0,0,0,113591.99,4036.67,0,109555.32,NaN
3457,2014,0163,Massachsuetts Toursim Fund,MASSACHUSETTS DEPARTMENT OF TRANSPORTATION,DOT,massachusetts department of transportation,70080900,Massachusetts Office of Travel and Tourism,0,0,...,0,0,0,0,0,5000,0,0,5000,NaN
3486,2017,0103,Commonwealth Transportation Fund,MASSACHUSETTS DEPARTMENT OF TRANSPORTATION,DOT,massachusetts department of transportation,15956368,Transfer from Commonwealth Transport Fnd to Ex...,0,0,...,NaN,0,0,0,0,0,0,0,0,NaN
4297,2014,0010,General Fund,MASSACHUSETTS DEPARTMENT OF TRANSPORTATION,DOT,massachusetts department of transportation,44001000,DEPARTMENT OF PUBLIC WELFARE,0,0,...,0,0,0,0,0,94918,72757.2,0,22160.8,NaN


Ok, I'll need to look up budget for mDOT, how much goes to MBT? Answer here https://budget.digital.mass.gov/govbudget/fy20/appropriations/transportation/

How does it show up in the expenditures dataset?

In [11]:
expenditures = client.get("pegc-naaa",where="budget_fiscal_year >= 2012", limit=999999)

In [12]:
expenditures = pd.DataFrame(expenditures)

In [13]:
pd.set_option("display.max.columns", 100)

In [14]:
expenditures[(expenditures["department"].str.contains("TRANSPORTATION"))  & 
            (expenditures["department"].str.contains("BAY"))]

,base_id,budget_fiscal_year,fiscal_period,date,cabinet_secretariat,department,appropriation_type,appropriation_name,object_class,object_code,encumbrance_id,zip_code,amount,fund,fund_code,appropriation_code,object,department_code,vendor,vendor_id,payment_id,payment_method,state,city,create_date


Ok, current idea is take % of payroll they spend on cops from cthru payroll and then apply that to each year as an estimate. Then do caculation that's like state police have x cops and y non-payroll expenses, that's z non-payroll expenses per cop and then apply that to MBTA

In [15]:
MBT_payroll = pd.DataFrame(client.get("rxhc-k6iz", 
                                  where = "department_division = 'Massachusetts Bay Transportation Authority (MBT)'",
                                  limit = 999999))


In [16]:
MBT_payroll["pay_total_actual"] = MBT_payroll["pay_total_actual"].astype(float)

In [41]:
MBT_payroll[MBT_payroll["position_title"].str.contains("Police")]

,year,name_last,name_first,department_division,position_title,position_type,service_end_date,pay_total_actual,pay_base_actual,pay_buyout_actual,pay_overtime_actual,pay_other_actual,annual_rate,pay_year_to_date,department_location_zip_code,contract,bargaining_group_no,bargaining_group_title,trans_no,chris,police_pay
14,2020,Haase,Mitchell,Massachusetts Bay Transportation Authority (MBT),"Officer,Police",Full Time Employee,2020-07-04T00:00:00.000,59197.160,34730.96,0,21422.19,3044.01,65915.2,59197.16,02118,MBTA Police Officers (M98),M98,MBTA Police Officers,19046690,MBT,59197.160
106,2020,Sanborn,Hayden,Massachusetts Bay Transportation Authority (MBT),"Officer,Police",Full Time Employee,2020-07-04T00:00:00.000,5627.740,5484.24,0,0,143.5,52811.2,5627.74,02118,MBTA Police Officers (M98),M98,MBTA Police Officers,19671314,MBT,5627.740
111,2020,Omwenga,Zack,Massachusetts Bay Transportation Authority (MBT),"Officer,Police",Full Time Employee,2020-07-04T00:00:00.000,41924.080,27870.16,0,11582.39,2471.53,64771.2,41924.08,02118,MBTA Police Officers (M98),M98,MBTA Police Officers,19252298,MBT,41924.080
196,2020,Bolotin,Michael,Massachusetts Bay Transportation Authority (MBT),"Officer,Police",Full Time Employee,2020-07-04T00:00:00.000,58023.960,37861.07,0,19028.51,1134.38,66040,58023.96,02118,MBTA Police Officers (M98),M98,MBTA Police Officers,16399305,MBT,58023.960
197,2020,Kerns,James,Massachusetts Bay Transportation Authority (MBT),"Lieutenant,Police",Full Time Employee,2020-07-04T00:00:00.000,90894.810,69615.83,0,15462.56,5816.42,121858.84,90894.81,02118,MBTA CLT Captain & Lieutenant (M96),M96,MBTA CLT Captain & Lieutenant,17245852,MBT,90894.810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29373,2019,HUNTER,CHRISTOPHER,Massachusetts Bay Transportation Authority (MBT),"Officer,Police",Full Time Employee,2019-10-26T00:00:00.000,25569.080,18777.99,0,5081.39,1709.7,0,25569.08,02118,MBTA Police Officers (M98),M98,MBTA Police Officers,18906134,MBT,25569.080
29397,2019,Diaz,Victor,Massachusetts Bay Transportation Authority (MBT),"Officer,Police",Full Time Employee,2019-12-07T00:00:00.000,9144.850,8281.08,0,554.28,309.49,67808,9144.85,02118,MBTA Police Officers (M98),M98,MBTA Police Officers,1521143,MBT,9144.850
29497,2018,Piacenza,Stephen,Massachusetts Bay Transportation Authority (MBT),Mgr Facilities - Police,Full Time Employee,2018-12-22T00:00:00.000,70863.220,70863.22,0,0,0,70863.179,70863.22,02118,MBTA L453 Professional (M05),M05,MBTA L453 Professional,2147912,MBT,70863.220
29508,2019,Smith,Chad,Massachusetts Bay Transportation Authority (MBT),"Officer,Police",Full Time Employee,2019-12-21T00:00:00.000,114115.940,97650.62,0,14603.91,1861.41,77604.8,114115.94,02118,MBTA Police Officers (M98),M98,MBTA Police Officers,2169908,MBT,114115.940


In [17]:
def get_police_pay(row):
    if "police" in row["position_title"].lower():
        return row["pay_total_actual"]
    else:
        return 0 

In [18]:
MBT_payroll["police_pay"]= MBT_payroll.apply(lambda x: get_police_pay(x), 
                                             axis =1 )

In [19]:
by_year = MBT_payroll.groupby("year").agg({"pay_total_actual":"sum", "police_pay":"sum"})

In [20]:
by_year.loc[:,"Percentage_Police_Pay"] = by_year.loc[:,"police_pay"]/by_year.loc[:,"pay_total_actual"]

In [21]:
by_year

,pay_total_actual,police_pay,Percentage_Police_Pay
year,,,
2017,327504051.310,15234666.310,0.047
2018,554244085.690,24217763.280,0.044
2019,586605881.310,31640635.490,0.054
2020,310488619.790,11782749.440,0.038


Ok so those numbers stay pretty constant. Why does it only go back to 2017?
Seems like we only have two years of full data from this dataset, ugh

In [22]:
MBT_payroll.groupby("year").size()

year
2017    6747
2018    7856
2019    7863
2020    7076
dtype: int64

Ok so next look at data from mDOT from this dataset, see if I can prefrom similar analysis

In [27]:
dept_name

'MASSACHUSETTS DEPARTMENT OF TRANSPORTATION (DOT)'

In [31]:
mDOT_payroll = pd.DataFrame(client.get("rxhc-k6iz", 
                                  where = "department_division = 'MASSACHUSETTS DEPARTMENT OF TRANSPORTATION (DOT)'",
                                  limit = 999999))


In [57]:
mDOT_payroll.groupby("year").size()

year
2010    5609
2011    6696
2012    5436
2013    5361
2014    5868
2015    5782
2016    5562
2017    4949
2018    4570
2019    5107
2020    4703
dtype: int64

In [55]:
print("mDOT payroll shape", mDOT_payroll.shape)
positions_by_size = mDOT_payroll.groupby("position_title").size().sort_values(ascending=False)

mDOT payroll shape (59643, 20)


In [56]:
positions_by_size.head(100)

position_title
Customer Service Rep I, RMV       3969
Civil Engineer III                3424
Maint Equipment Operator I        2922
Civil Engineer I                  2424
Civil Engineer II                 2113
Civil Engineer IV                 1944
Toll Collector I                  1905
Genl Construction Inspector II    1735
Contracted Student Interns        1733
Toll Collector-Seasonal           1714
Genl Construction Inspector I     1497
Customer Service Rep II, RMV      1431
Civil Engineer V                  1249
Local 127 - Retired - SLB         1214
Program Coordinator III           1203
Toll Collector I PT                900
Civil Engineer VI                  870
Program Coordinator II             844
Customer Service Rep III, RMV      739
Sys Programmer/Sys Supv, Pdpp      704
Highway Maint Foreman III          645
Toll Collector                     628
Local 5696 USW - Retired - SLB     586
Engineering Aide II                565
DOT Non Union - Retired - SLB      558
Administra

In [35]:
mDOT_payroll[mDOT_payroll["position_title"].str.contains("Police")]["position_title"].unique()

array(['State Police Dispatcher II', 'State Police Dispatcher III',
       'State Police Trooper,1st Class', 'State Police Sergeant',
       'Police Statistical Clerk', 'State Police Trooper'], dtype=object)

In [58]:
mDOT_payroll[mDOT_payroll["position_title"].str.contains("Officer")]["position_title"].unique()

array(['Compliance Officer III', 'Compliance Officer I',
       'Title Review Officer', 'Administrative Rev Officer II',
       'Compliance Officer II', 'Information Officer I',
       'Information Officer II', 'Information Officer III',
       'Chief Compliance Officer', 'Administrative Rev Officer I',
       'Admin Review Officer II', 'Chief Administrative Officer',
       'Personnel Officer II', 'Personnel Officer I',
       'ActingChief ProcurementOfficer', 'Fiscal Officer VI',
       'Chief Financial Officer', 'Fiscal Officer VIII',
       'Affirmative Action Officer', 'Chief Security Officer',
       'Chief Operating Officer', 'Chief Strategic Officer',
       'Affirmative Market Officer', 'Fiscal Officer/MFO',
       'Deputy Chief Info Officer', 'Highway Chief Admin Officer',
       'Fiscal Officer VII', 'Deputy Chief Operating Officer',
       'Dpty Chief Diversity Officer', 'Chief Procurement Officer',
       'Chief Information Officer', 'Dep.Chief Technology Officer',
       

Ok, so this data doesn't seem to include the MBTA cops.
Next thing to try: I found a 2017 payroll pdf for the MBTA which seems to include the MBTA cops. See if those numbers are in line with what is in cthru in total amounts and % of payroll that goes to police pay in cthru data

In [60]:
from PyPDF2 import PdfFileReader

In [78]:
pdf_path = "data/MBTA_pdfs/mbta-payroll-2017.pdf"

In [61]:
mbta_2017 = PdfFileReader("data/MBTA_pdfs/mbta-payroll-2017.pdf")

In [69]:
mbta_2017.getPage(6).extractText()

'@#=@@&47B,\'L()\n+*7B1N\nK1B4\')*4>3:F/IL4M\'GB\n%@:&L5:%"\n<!%>$;!?88\n<!9>=$$?!;\n@#9"#&47B,B\'L()\nZJ\',\n.\'GN2,27)\n#8:KG):%!\n<=#>#8%?;8\n<$9>=%8?$8\n@%;@!&7G*(2((*\nD*L2B\nZB1\'24B4>D2,B/:/%7)/0(\'77\n#;:&L5:%9\n<$!>9$8?!8\n<%"9>%##?;"\n@#9%9&7NEJ\n0(B*1\')4\'\nK1B4\')*4>3:F/IL4M\'GB\n#$:IB1:%!\n<!@>"!=?=8\n<"#>99;?;$\n@9";#&7N6\',\nF4BG2\'\n&,\'(J7)>34*\\BG)/0*,)4*(7\n%@:KG):%;\n<%88>89;?8!\n<$=>#@$?"!\n;$""9&7N6\',\nZ\'GN\'B(\n.*)*41B47*,>Z\'12Q/F4\',72)\n89:.\'4:8=\n<@@>";9?#8\n<=!>;88?@8\n@9;!9&72Q\'N\nOE4\'N26\nW,52,BB4\n8;:PBE:%@\n<@=>;!!?=8\n<=9>%8"?"@\n;"==9&)N\',\'72*L\nF42M*,\nP*4B1B47*,>&L)*/.)GB\n#$:.\'J:8%\n<$@>"9%?#8\n<%9%>%!!?9;\n;@@$$&)N\',\'72*L\nX211*G4\')B7\n.\'GN2,27)\n%9:+L,:8"\n<=#>#8%?;8\n<=$>#$;?";\n!@@;;@&)2(\nX2)(B4\nK1B4\')*4>3:F/IL4M\'GB\n%$:S*C:%@\n<98>@;9?#8\n<#>#88?88\n;;;!$&)H2,7/-\'6E(B\nI)\'G2\n.*)*41B47*,>I)4BB)G\'4\n#%:KG):8#\n<@">99@?;8\n<=">;"%?%%\n;$%$;&)H2,7*,\n&,)N*,J\n.*)*41B47*,>Z\'12Q/F4\',72)\n%8:IB1:8@\n<@@>";9?#8\n<=$>%@=?!=\n@#8%"

In [74]:
pdf_FileObj= open(, 'rb')

mbta_2017 = PdfFileReader(pdf_FileObj)
mbta_2017

In [76]:
mbta_2017.getPage(0).extractText()

'!"#$%&\'\'()*\n+,-.(/,"\'\n012-.(/,"\'\n31.$\'\n412\'(*,.\'\n5,-\'(6,&\n3%.,$(789:(!,2;1;<-\n!"#$%#&\'()*\n+*\',,\'\n&-./0\'12)\'(/34*54\'67\n89:&14:%;\n<%=">888?88\n<#=>!;%?""\n@#$#$&\'4*,\nABC\'4\nD\'E*4B4>F4\'GH\n#%:IB1:%"\n<"@>=89?#8\n<@#>=88?;9\n@#8#%&\'4*,\nABJB,\'\nK1B4\')*4>3:F/IL4M\'GB\n8":.\'J:%!\n<!@>"!=?=8\n<!=>!!=?=%\n;=%8%&E\',J\nI\'6LB(\nIB45B\',)\n%8:&14:8;\n<$">"%=?!8\n<#8#>#%!?%$\n;=!%"&EE2)\n0J,)N2\'\nO,71BG)*4\n8":IB1:8;\n<=9>98!?88\n<$=>@88?#=\n$@!$&EE*))\nAB,,B)N\nK1B4\')*4>IL4M\'GB\n89:PBE:$#\n<@!>"==?=8\n<=">;9%?;=\n@#%89&EE*))\n0N\'4(B7\nK1B4\')*4>IL4M\'GB\n#9:+L,:%!\n<;9>9$=?!8\n<@!>#98?!;\n@9;#"&EE*))\n0*4BJ\nK1B4\')*4>3:F/IL4M\'GB\n8$:+\',:%@\n<98>@;9?#8\n<#$>;8=?!;\n!"%!#@&EQB(H\'426\nD*\'J.54/RB725,\n#@:PBE:%#\n<%%@>#@$?$$\n<%%;>9"!?9=\n@%=%8&EQB((\'\nS\'724\nK1B4\')*4>IL4M\'GB\n8;:+\',:%!\n<;@>%#%?;8\n<@9>"!#?$$\n@#@9!&EQB(*L\'NN\'E\n&(2\nK1B4\')*4>IL4M\'GB\n##:+L,:%"\n<;9>9$=?!8\n<;9>;#!?#%\n;@$;"&EQB(4\'N6\',\n&7N4\'M\nIL1C/F4\',7/:/TL7\n%%:KG):8"\n<$%

In [79]:
mbta_2017 = textract.process(pdf_path, method="tesseract", language="eng")

In [ ]:
mbta_2017 = mbta_2017.replace("//n", " ")

In [ ]:
mbta_2017